# Code-along 2024-02-27 Analyzing Google Sheets Data in Python

In this code-along, we will be analyzing Google Analytics data. The sample dataset contains obfuscated GA360 data for August 1, 2017 from the [Google Merchandise Store](https://shop.googlemerchandisestore.com/), a real ecommerce store selling Google branded merchandise ([Source](https://support.google.com/analytics/answer/7586738)).

This data is typical for what you'd see for an ecommerce website. It contains session data with traffic source, location and transcation info. Other data has been anonymized. The data is available for in a publicly available Google Sheet. We will be using a combination of SQL and Python to analyze this dataset, getting more insights into what is driving sales on this particular day.

## Task 0: Setup

- Make sure you're signed into Google.
- Click [this link](https://docs.google.com/spreadsheets/d/1VmKJ94bk2yLtkYauHAuCaRV2xZ1KV4H_vUYMq0fPo6Q/copy) to copy a Google Sheet with the data to your own Google account.

## Task 1: Import the session data

In the SQL cell below, click "Connect a database", click "Google Sheets", follow the instructions, and select your copy of the Google Sheets file. Use a SQL statement to pull in all the data from the Google Sheet.

In [110]:
SELECT * FROM 'ga_sessions'

,visitId,visitStartTime,visitNumber,channelGrouping,browser,operatingSystem,deviceCategory,continent,subContinent,country,transactions
0,1501591568,1501591568,1,Organic Search,Chrome,Windows,desktop,Europe,Southern Europe,Greece,NaN
1,1501589647,1501589647,2,Referral,Chrome,Windows,desktop,Asia,Southern Asia,India,NaN
2,1501616621,1501616621,1,Referral,Chrome,Windows,desktop,Europe,Northern Europe,United Kingdom,NaN
3,1501601200,1501601200,1,Referral,Firefox,Windows,desktop,Americas,Northern America,United States,NaN
4,1501615525,1501615525,1,Referral,Chrome,Windows,desktop,Americas,Northern America,United States,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2551,1501638264,1501638264,1,Referral,Chrome,Macintosh,desktop,Americas,Northern America,United States,NaN
2552,1501605330,1501605330,1,Organic Search,Chrome,Macintosh,desktop,Americas,Northern America,United States,NaN
2553,1501615026,1501615026,3,Organic Search,Chrome,Macintosh,desktop,Americas,Northern America,United States,NaN
2554,1501627131,1501627131,16,Organic Search,Chrome,Windows,desktop,Americas,Northern America,United States,1.0


## Task 2: Import packages

We'll use `pandas` and `plotly`, so let's pull these in.

In [111]:
# Import packages
import pandas as pd
import plotly.express as px

## Task 3: Clean the data

Let's inspect the types of the columns, and make adjustments where needed.

In [112]:
df.dtypes

visitId              int64
visitStartTime       int64
visitNumber          int64
channelGrouping     object
browser             object
operatingSystem     object
deviceCategory      object
continent           object
subContinent        object
country             object
transactions       float64
dtype: object

In [113]:
#creating a copy of the data
ga_df=df.copy()

In [114]:
#converting the visitStartTime from an integer to a date
ga_df['visitStartTime']=pd.to_datetime(ga_df['visitStartTime'],unit='s')
ga_df.head()

,visitId,visitStartTime,visitNumber,channelGrouping,browser,operatingSystem,deviceCategory,continent,subContinent,country,transactions
0,1501591568,2017-08-01 12:46:08,1,Organic Search,Chrome,Windows,desktop,Europe,Southern Europe,Greece,NaN
1,1501589647,2017-08-01 12:14:07,2,Referral,Chrome,Windows,desktop,Asia,Southern Asia,India,NaN
2,1501616621,2017-08-01 19:43:41,1,Referral,Chrome,Windows,desktop,Europe,Northern Europe,United Kingdom,NaN
3,1501601200,2017-08-01 15:26:40,1,Referral,Firefox,Windows,desktop,Americas,Northern America,United States,NaN
4,1501615525,2017-08-01 19:25:25,1,Referral,Chrome,Windows,desktop,Americas,Northern America,United States,NaN


## Task 4: Explore the data

Use the predefined plotting function to explore different grouped session counts as either a bar chart or a pie chart.

In [115]:
def plot_sessions_per_group(df, group, viz_type='bar'):
    sessions_per_group = df.groupby(group).size().reset_index(name='sessions').sort_values(by='sessions')
    if viz_type == 'bar':
        return px.bar(sessions_per_group,
                      x=group,
                      y='sessions',
                      title=f'Number of sessions per {group}',
                      text='sessions')
    elif viz_type == 'pie':
        return px.pie(sessions_per_group,
                      names=group,
                      values='sessions',
                      title=f'Distribution of sessions per {group}')
    else:
        raise ValueError("viz_type can only be 'bar' or 'pie'")

In [116]:
#Number of sessions per browser
plot_sessions_per_group(ga_df,'browser','bar')

In [117]:
#Number of sessions per channel group
plot_sessions_per_group(ga_df,'channelGrouping','pie')

In [118]:
#Number of sessions per operating system
plot_sessions_per_group(ga_df,'operatingSystem','bar')

In [119]:
#Number of sessions per device category
plot_sessions_per_group(ga_df,'deviceCategory','pie')

In [120]:
#Number of sessions per continent
plot_sessions_per_group(ga_df,'continent','pie')

In [121]:
#Number of sessions for top 10 countries
top_10_df=ga_df.groupby('country',as_index=False).agg({'visitId':'count'}).sort_values(by='visitId',ascending=False).head(10)
px.bar(top_10_df,
                      x='country',
                      y='visitId',
                      title=f'Number of sessions for top 10 countries',
                      text='country')

## Task 5: Show hourly visits per channelGrouping

Adjust the code below to show the hourly visits throughout the day per `channelGrouping`.

In [122]:
#hourly visits per channelGrouping 
#the commented line is used to convert the visitStartTime to us time 
#ga_df['visitStartTime'] = ga_df['visitStartTime'].dt.tz_convert('America/Los_Angeles')
ga_df['hour'] = ga_df['visitStartTime'].dt.to_period('H')
visits_per_month = ga_df.groupby(['channelGrouping','hour']).size().reset_index(name='visits')
visits_per_month['hour'] = visits_per_month['hour'].astype(str)
px.line(visits_per_month, x='hour', y='visits',color='channelGrouping', title='Visits per hour throughout the day for each channel', markers=True)

In [123]:
#Visits per hour throughout the day for all types
#ga_df['visitStartTime'] = ga_df['visitStartTime'].dt.tz_convert('America/Los_Angeles')
ga_df['hour'] = ga_df['visitStartTime'].dt.to_period('H')
visits_per_month = ga_df.groupby(['hour']).size().reset_index(name='visits')
visits_per_month['hour'] = visits_per_month['hour'].astype(str)
px.line(visits_per_month, x='hour', y='visits', title='Visits per hour throughout the day ', markers=True)

## Task 6 (Extra): which channel leads to most transactions per session?

In [124]:
#Ratio of transactions per channelGrouping
#grouping and counting in the dataframe
cg_stats = (
    ga_df
    .groupby('channelGrouping')
    .agg({'transactions': 'sum', 'visitId': 'size'})
    .reset_index()
)
# Calculate the ration of transactions per session column
cg_stats['ratio']=cg_stats['transactions']/cg_stats['visitId']
#sorting the cg_stats df
cg_stats=cg_stats.sort_values(by='ratio')
# visualising the data on a bar chart
px.bar(cg_stats,x='channelGrouping',y='ratio',title='Ratio of transactions per channelGrouping')